In [2]:
import json

# Load the OpenAPI JSON file
with open("plaid-openapi.json", "r") as file:
    openapi_data = json.load(file)

def get_properties_from_ref(ref, components):
    """
    Given a $ref string, extract the actual schema it points to.
    """
    schema_name = ref.replace("#/components/schemas/", "")
    return components["schemas"][schema_name]


def merge_allOf_schemas(allOf, components):
    """
    Merge schemas under "allOf" into a single schema.
    """
    merged_schema = {"properties": {}}
    for subschema in allOf:
        if "$ref" in subschema:
            subschema = get_properties_from_ref(subschema["$ref"], components)
        if "properties" in subschema:
            merged_schema["properties"].update(subschema["properties"])
    return merged_schema


def extract_properties_from_allOf(allOf, components, prefix=""):
    """
    Extract properties from a given "allOf" list.
    """
    properties = []
    for subschema in allOf:
        properties += extract_properties(subschema, components, prefix)
    return properties


def extract_properties(schema, components, prefix=""):
    """
    Recursively extract properties from a given schema.
    """
    properties = []

    # If the schema is a reference, dereference it
    if "$ref" in schema:
        schema = get_properties_from_ref(schema["$ref"], components)

    # If the schema has "allOf", extract properties from each of the schemas within
    if "allOf" in schema:
        properties += extract_properties_from_allOf(schema["allOf"], components, prefix)

    # If the schema is of type object and has properties, iterate over them
    if schema.get("type") == "object" and "properties" in schema:
        for prop_name, prop_schema in schema["properties"].items():
            full_name = f"{prefix}.{prop_name}" if prefix else prop_name
            properties += extract_properties(prop_schema, components, full_name)
    elif schema.get("type") == "array" and "items" in schema:
        properties += extract_properties(schema["items"], components, prefix)
    else:
        prop_details = {
            "property_name": prefix,
            "property_type": schema.get("type", "N/A"),
            "property_description": schema.get("description", "N/A"),
            "enum": schema.get("enum", []),
            "nullable": schema.get("nullable", False)
        }
        properties.append(prop_details)

    return properties


endpoint_path = "/accounts/get"
response_schema = openapi_data["paths"][endpoint_path]["post"]["responses"]["200"]["content"]["application/json"]["schema"]
properties_list = extract_properties(response_schema, openapi_data["components"])

properties_list

[{'property_name': 'accounts.account_id',
  'property_type': 'string',
  'property_description': "Plaid’s unique identifier for the account. This value will not change unless Plaid can't reconcile the account with the data returned by the financial institution. This may occur, for example, when the name of the account changes. If this happens a new `account_id` will be assigned to the account.\n\nThe `account_id` can also change if the `access_token` is deleted and the same credentials that were used to generate that `access_token` are used to generate a new `access_token` on a later date. In that case, the new `account_id` will be different from the old `account_id`.\n\nIf an account with a specific `account_id` disappears instead of changing, the account is likely closed. Closed accounts are not returned by the Plaid API.\n\nLike all Plaid identifiers, the `account_id` is case sensitive.",
  'enum': [],
  'nullable': False},
 {'property_name': 'accounts.balances.available',
  'proper

In [10]:
def render_as_markdown_table(properties_list):
    """
    Convert a list of properties to a markdown table.
    """
    # Header for the markdown table
    header = "| Property Name | Type | Description | Enum | Nullable |\n"
    header += "|---------------|------|-------------|------|----------|\n"

    # Generate rows for the table
    rows = []
    for prop in properties_list:
        enum_values = ", ".join(prop["enum"]) if prop["enum"] else "None"
        row = f"| {prop['property_name']} | {prop['property_type']} | {prop['property_description']} | {enum_values} | {prop['nullable']} |"
        rows.append(row)

    return header + "\n".join(rows)

# Render the properties list as a markdown table
markdown_table = render_as_markdown_table(properties_list)

# Display the first few lines of the markdown table for verification
markdown_table.split("\n")

['| Property Name | Type | Description | Enum | Nullable |',
 '|---------------|------|-------------|------|----------|',
 "| accounts.account_id | string | Plaid’s unique identifier for the account. This value will not change unless Plaid can't reconcile the account with the data returned by the financial institution. This may occur, for example, when the name of the account changes. If this happens a new `account_id` will be assigned to the account.",
 '',
 'The `account_id` can also change if the `access_token` is deleted and the same credentials that were used to generate that `access_token` are used to generate a new `access_token` on a later date. In that case, the new `account_id` will be different from the old `account_id`.',
 '',
 'If an account with a specific `account_id` disappears instead of changing, the account is likely closed. Closed accounts are not returned by the Plaid API.',
 '',
 'Like all Plaid identifiers, the `account_id` is case sensitive. | None | False |',
 

In [3]:
from IPython.display import display
import pandas as pd
def display_as_table(properties_list):
    """
    Display a list of properties as a table in a Jupyter notebook cell.
    """
    # Convert the properties list to a DataFrame
    df = pd.DataFrame(properties_list)

    # Display the DataFrame as a table in the Jupyter notebook cell
    display(df)

# Display the properties list as a table in the Jupyter notebook cell
display_as_table(properties_list)

,property_name,property_type,property_description,enum,nullable
0,accounts.account_id,string,Plaid’s unique identifier for the account. Thi...,[],False
1,accounts.balances.available,number,The amount of funds available to be withdrawn ...,[],True
2,accounts.balances.current,number,The total amount of funds in or owed by the ac...,[],True
3,accounts.balances.limit,number,"For `credit`-type accounts, this represents th...",[],True
4,accounts.balances.iso_currency_code,string,The ISO-4217 currency code of the balance. Alw...,[],True
5,accounts.balances.unofficial_currency_code,string,The unofficial currency code associated with t...,[],True
6,accounts.balances.last_updated_datetime,string,Timestamp in [ISO 8601](https://wikipedia.org/...,[],True
7,accounts.mask,string,The last 2-4 alphanumeric characters of an acc...,[],True
8,accounts.name,string,"The name of the account, either assigned by th...",[],False
9,accounts.official_name,string,The official name of the account as given by t...,[],True


In [4]:
example_json = {
    "accounts": [
        {
            "account_id": "blgvvBlXw3cq5GMPwqB6s6q4dLKB9WcVqGDGo",
            "balances": {
                "available": 100,
                "current": 110,
                "iso_currency_code": "USD",
                "limit": None,
                "unofficial_currency_code": None
            },
            "mask": "0000",
            "name": "Plaid Checking",
            "official_name": "Plaid Gold Standard 0% Interest Checking",
            "persistent_account_id": "8cfb8beb89b774ee43b090625f0d61d0814322b43bff984eaf60386e",
            "subtype": "checking",
            "type": "depository"
        },
        {
            "account_id": "6PdjjRP6LmugpBy5NgQvUqpRXMWxzktg3rwrk",
            "balances": {
                "available": None,
                "current": 23631.9805,
                "iso_currency_code": "USD",
                "limit": None,
                "unofficial_currency_code": None
            },
            "mask": "6666",
            "name": "Plaid 401k",
            "official_name": None,
            "subtype": "401k",
            "type": "investment"
        },
        {
            "account_id": "XMBvvyMGQ1UoLbKByoMqH3nXMj84ALSdE5B58",
            "balances": {
                "available": None,
                "current": 65262,
                "iso_currency_code": "USD",
                "limit": None,
                "unofficial_currency_code": None
            },
            "mask": "7777",
            "name": "Plaid Student Loan",
            "official_name": None,
            "subtype": "student",
            "type": "loan"
        }
    ],
    "item": {
        "available_products": [
            "balance",
            "identity",
            "payment_initiation",
            "transactions"
        ],
        "billed_products": [
            "assets",
            "auth"
        ],
        "consent_expiration_time": None,
        "error": None,
        "institution_id": "ins_117650",
        "item_id": "DWVAAPWq4RHGlEaNyGKRTAnPLaEmo8Cvq7na6",
        "update_type": "background",
        "webhook": "https://www.genericwebhookurl.com/webhook"
    },
    "request_id": "bkVE1BHWMAZ9Rnr"
}

def get_example_for_property(example_json, property_path):
    """
    Extracts an example value for a given property path from the example json.
    """
    parts = property_path.split('.')
    try:
        for part in parts:
            if isinstance(example_json, list):
                example_json = example_json[0]
            example_json = example_json[part]
        return example_json
    except (KeyError, TypeError):
        return None

def append_examples_to_properties(properties, example_json):
    """
    Appends an example to each property in the list of properties based on the example json.
    """
    for prop in properties:
        prop["example"] = get_example_for_property(example_json, prop["property_name"])
    return properties

# Add example data to properties
properties_with_examples = append_examples_to_properties(properties_list, example_json)

properties_with_examples

[{'property_name': 'accounts.account_id',
  'property_type': 'string',
  'property_description': "Plaid’s unique identifier for the account. This value will not change unless Plaid can't reconcile the account with the data returned by the financial institution. This may occur, for example, when the name of the account changes. If this happens a new `account_id` will be assigned to the account.\n\nThe `account_id` can also change if the `access_token` is deleted and the same credentials that were used to generate that `access_token` are used to generate a new `access_token` on a later date. In that case, the new `account_id` will be different from the old `account_id`.\n\nIf an account with a specific `account_id` disappears instead of changing, the account is likely closed. Closed accounts are not returned by the Plaid API.\n\nLike all Plaid identifiers, the `account_id` is case sensitive.",
  'enum': [],
  'nullable': False,
  'example': 'blgvvBlXw3cq5GMPwqB6s6q4dLKB9WcVqGDGo'},
 {'pr

In [5]:
def save_to_html(properties_list, filename):
    """
    Render a list of properties as an HTML table and save to a file.
    """
    # Convert the properties list to a DataFrame
    df = pd.DataFrame(properties_list)

    # Save the DataFrame as an HTML file
    df.to_html(filename, index=False, escape=False)

# Save the properties list as an HTML file
save_to_html(properties_with_examples, "plaid_accounts_get.html")


In [43]:
display_as_table(properties_list)

,property_name,property_type,property_description,enum,nullable,example
0,accounts.account_id,string,Plaid’s unique identifier for the account. Thi...,[],False,BxBXxLj1m4HMXBm9WZZmCWVbPjX16EHwv99vp
1,accounts.balances.available,number,The amount of funds available to be withdrawn ...,[],True,110
2,accounts.balances.current,number,The total amount of funds in or owed by the ac...,[],True,110
3,accounts.balances.limit,number,"For `credit`-type accounts, this represents th...",[],True,None
4,accounts.balances.iso_currency_code,string,The ISO-4217 currency code of the balance. Alw...,[],True,USD
...,...,...,...,...,...,...
77,item.products,string,A list of products that an institution can sup...,"[assets, auth, balance, identity, investments,...",False,None
78,item.consented_products,string,A list of products that an institution can sup...,"[assets, auth, balance, identity, investments,...",False,None
79,item.consent_expiration_time,string,The RFC 3339 timestamp after which the consent...,[],True,None
80,item.update_type,string,Indicates whether an Item requires user intera...,"[background, user_present_required]",False,background


In [6]:
import json

def write_properties_to_file(properties, filename):
    """
    Writes the properties list to a file.
    """
    with open(filename, 'w') as f:
        json.dump(properties, f, indent=4)

# Testing the functions
write_properties_to_file(properties_with_examples, "plaid_accounts_get.json")